### Comba, Mateo Carlos. 

### Parcial Aprendizaje Automatico I.

### Ejercicio: descubrir cuál es el modelo que mejor pronostica la variable quality. Caso: vino tinto.

Primero, importo las librerías necesarias para trabajar: Numpy, Pandas, Matplotlib y Scikit Learn. De ésta última, los modelos a probar en nuestro problema de clasificación: arbol de decisiones, kNN y regresión logística.

In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Librerías a utilizar en la selección de modelos
from sklearn import linear_model        # Para regresión Logística
from sklearn import neighbors           # para kNN
from sklearn.tree import DecisionTreeClassifier  # para Árbol de Decisión

# Librerías para evaluar
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

### Importo los datos

In [129]:
df=pd.read_csv('winequality-red.csv', sep = ";") #Estan separados por ; y o por , por eso le agrego el separador correcto
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Exploro los datos para ver con que me encuentro...

In [130]:
df.info()
#Tengo 1599 registros no nulos, 12 columnas, de las cuales sé que el target es la columna "quality", la base se ve limpia

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [131]:
df.describe() #No se ve nada raro por aqui tampoco... todas variables continuas las que usaremos para pronosticar

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


### Division entre train y test set

In [132]:
X=df.drop(axis=1,columns='quality')  # En esta variable guardo todas variables que utilizo para pronosticar
y=df['quality']                      # guardamos acá la variable taget quality

Voy a dividir los datos con el metodo train_test_split, con un tamaño de test de 20% de los datos

In [133]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)

Vuelvo a dividir los datos para obtener los validation, ya que los necesito para hacer la seleccion de modelos

In [134]:
X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)

Ahora, entrenaremos y evaluaremos cada modelo en el validation, obteniendo los mejores hiperparámetros para cada uno.

Luego, compararemos y haremos la selección del mejor.

### Selección de  Modelo e hiperparámetros:

#### Regresión Logística

El modelo de regresión Logística es no paramétrico, así que directamente lo probamos.

In [135]:
#Creo el modelo
modeloRLog=linear_model.LogisticRegression(penalty='none',fit_intercept=True, random_state=123)
#Lo entrenamos con el metodo fit
modeloRLog.fit(X_val_train,y_val_train)
#Intente aumentar las iteraciones, pero no se va el tema del warning, no supe como eliminarlo asique le agregue un warning ignore.

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=20000,
                   multi_class='auto', n_jobs=None, penalty='none',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [136]:
#Creo la variable y_val_pred con las predicciones que el modelo hace sobre los datos del validation test
y_val_pred=modeloRLog.predict(X_val_test)

#Evaluamos en el validation test, las predicciones que realizó el modelo guardadas en la variable y_val_pred
AC_RLog=round(accuracy_score(y_val_test, y_val_pred),3)
F1_RLog=round(f1_score(y_val_test,y_val_pred, average='macro'),3)

# Mostrammos:

print("Evaluación modelo de Regresión Logística en el Validation")
print("Accuracy: ", AC_RLog)
print("F1: ", F1_RLog)

Evaluación modelo de Regresión Logística en el Validation
Accuracy:  0.617
F1:  0.273


#### kNN

Para kNN, debemos seleccionar el mejor hiperparámetro "K", es decir, la cantidad de puntos vecinos que tenemos en cuenta en las cercanías para clasificar los grupos (que ya se conocen de antemano).

In [137]:
k_valores=np.arange(1,101,1)   # voy a probar los K desde 1 a 100, por esto creo el array que luego voy a iterar
k_valores                     

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

Para cada valor de K, voy a crear el modelo, entrenarlo y evaluarlo con F1 y Accuracy, tomando este último como el criterio para definir cual es el mejor K. Si el valor de Accuracy obtenido por el K es mejor que el anterior, lo voy a guardar en una variable para mostrar al final de las 50 iteraciones cual fue el mejor resultado y a que K corresponde.

In [138]:
print("Evaluación modelo kNN en el Validation")

#Variables auxiliares donde se almacenarán los resultados del modelo elegido, las seteamos en 0 para empezar.
k_max=0
F1kNN_max=0
ACkNN_max=0

for k in k_valores:
    modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='uniform') #Creo modelo
    modelokNN.fit(X_val_train, y_val_train)                                     #Entreno
    y_val_pred=modelokNN.predict(X_val_test)                                    #Pronostico
    ACkNN=accuracy_score(y_val_test, y_val_pred)                                #Obtengo AC
    F1kNN=f1_score(y_val_test,y_val_pred, average='macro')                      #Obtengo F1
    if ACkNN>ACkNN_max:                 #Comparo Accuracy con el maximo y si es mayor, lo reemplazo.
        k_max=k                                                                 
        F1kNN_max=round(F1kNN,3)
        ACkNN_max=round(ACkNN,3)
    
print('Valor de k: ', k_max)                  #Hago print del mejor K y su respectivo Accuracy y F1.
print("Accuracy: ", ACkNN_max)
print("F1: ", F1kNN_max)

Evaluación modelo kNN en el Validation
Valor de k:  35
Accuracy:  0.578
F1:  0.228


#### Arbol de decisión

Los arboles de decision, tienen el hiperparámetro "max-depth", que significa la profunidad del arbol. Siguiendo un criterio similar al anterior, vamos a iterar para obtener la profundidad que nos provea el mejor AC, sin caer en overfitting.

In [139]:
#Igual que en el modelo anterior, creo array de numeros del 1 al 100 para luego iterar.
profundidades=np.arange(1,201,1)

In [140]:
#Creo variables auxiliares donde se almacenarán los resultados del modelo elegido, las seteamos en 0 para empezar.

p_max=0               #Aca guardaremos la profundidad del arbol que mejor resuelve el problema (mayor AC)
ACT_max=0
F1T_max=0

print("Evaluación modelo Árbol de Decisión en el Validation")

for p in profundidades: 
    modeloT=DecisionTreeClassifier(max_depth=p, random_state=123)        #Creamos el modelo con profundidad p
    modeloT.fit(X_val_train, y_val_train)                                # Lo entrenamos
    y_val_pred = modeloT.predict(X_val_test)                             # Pronosticamos los valores de y para los valores del X_test
    ACT=accuracy_score(y_val_test, y_val_pred)
    F1T=f1_score(y_val_test,y_val_pred, average='macro')
    
    if ACT>ACT_max:                                                      #Utilizo accuracy como el parámetro que decide.
        p_max=p
        F1T_max=round(F1T,3)
        ACT_max=round(ACT,3)
    
print('Valor de profundidad ', p_max)
print("Accuracy: ", ACT_max)
print("F1: ", F1T_max)

Evaluación modelo Árbol de Decisión en el Validation
Valor de profundidad  1
Accuracy:  0.598
F1:  0.215


#### Comparacion de resultados

Comparamos los resultados y elegimos el modelo a llevar a producción. Métrica utilizada: Accuracy.

In [141]:
#Creo DataFrame con los valores obtenidos de cada modelo
dfRtdos= pd.DataFrame({'Modelo': ['Reg Log', 'kNN', 'Arbol'], 'Accuracy': [AC_RLog, ACkNN, ACT], 'F1': [F1_RLog, F1kNN, F1T], 'Hiperparametros': [ np.nan , k_max, p_max]})
#Pongo los modelos como indice
dfRtdos = round(dfRtdos.set_index('Modelo'),3)
dfRtdos

,Accuracy,F1,Hiperparametros
Modelo,,,
Reg Log,0.617,0.273,NaN
kNN,0.512,0.185,35.0
Arbol,0.559,0.271,1.0


De ese DataFrame, elijo el modelo con el mejor Accuracy y lo muestro

In [142]:
print('Finalmente, el modelo seleccionado es: ')

dfRtdos.loc[[(dfRtdos['Accuracy'].idxmax())]]

Finalmente, el modelo seleccionado es: 


,Accuracy,F1,Hiperparametros
Modelo,,,
Reg Log,0.617,0.273,NaN


Muy bien, ahora ya sabemos cual es el modelo seleccionado, evaluémoslo entrando con el Train set vs el Test set.


In [143]:
modeloRLog2=linear_model.LogisticRegression(max_iter= 20000,penalty='none',fit_intercept=True, random_state=123)
modeloRLog2.fit(X_train, y_train)              # Lo entrenamos
y_pred = modeloRLog2.predict(X_test)           # Pronosticamos los valores de y para los valores del X_test
ACRLog_final=accuracy_score(y_test, y_pred)
F1RLog_final=f1_score(y_test,y_pred, average='macro')

print("Evaluación del modelo Seleccionado (Comportamiento esperado en el mundo real): ")
print("Accuracy: ", round(ACRLog_final,3))
print("F1: ", round(F1RLog_final,3))

Evaluación del modelo Seleccionado (Comportamiento esperado en el mundo real): 
Accuracy:  0.525
F1:  0.233


### Paso a producción:

Finalmente el modelo que llevaríamos a producción sería modelRLog2, pero podemos aprovechar todos los datos que tenemos para crear el modelo siempre que dejemos el valor de F1 y AC obtenidos con anterioridad como los estimadores de lo que ocurrirá en "la realidad", es decir los casos totalmente nuevos:

In [144]:
#Creamos el modelo de Regresion Logistica
modelo_produccion=linear_model.LogisticRegression(max_iter= 20000,penalty='none',fit_intercept=True, random_state=123)
# lo entrenamos con todo el dataset original
modelo_produccion.fit(X, y)                                                     

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=20000,
                   multi_class='auto', n_jobs=None, penalty='none',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

En este momento, el modelo está listo para recibir nuevas observaciones y resolver el problema inicial (pronosticar la variable quality, en base a las otras features)